In [1]:
# Import all neccesary modules for cleaning
import pandas as pd
import matplotlib.pyplot as plt
import warnings

In [2]:
warnings.filterwarnings("ignore") # Low memory - diff types warning
df_p2p = pd.read_csv("data/case_study_loan_FS.csv")
df_p2p.head(3)

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,1,2.014123e+09,5056179,10400.0,10400.0,10400.0,36 months,6.99%,321.08,A,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2.014123e+09,4975197,15000.0,15000.0,15000.0,60 months,12.39%,336.64,C,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2.014123e+09,6343655,12800.0,12800.0,12800.0,60 months,17.14%,319.08,D,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_p2p.info('''verbose=True''')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235631 entries, 0 to 235630
Data columns (total 145 columns):
Unnamed: 0                                    int64
id                                            float64
member_id                                     int64
loan_amnt                                     float64
funded_amnt                                   float64
funded_amnt_inv                               float64
term                                          object
int_rate                                      object
installment                                   float64
grade                                         object
sub_grade                                     object
emp_title                                     object
emp_length                                    object
home_ownership                                object
annual_inc                                    float64
verification_status                           object
issue_d                             

In [4]:
df_p2p

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,1,2.014123e+09,5056179,10400.0,10400.0,10400.0,36 months,6.99%,321.08,A,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2.014123e+09,4975197,15000.0,15000.0,15000.0,60 months,12.39%,336.64,C,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2.014123e+09,6343655,12800.0,12800.0,12800.0,60 months,17.14%,319.08,D,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2.014123e+09,6060755,9600.0,9600.0,9600.0,36 months,13.66%,326.53,C,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2.014123e+09,5544167,21425.0,21425.0,21425.0,60 months,15.59%,516.36,D,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
5,6,2.014123e+09,5060568,7650.0,7650.0,7650.0,36 months,13.66%,260.20,C,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
6,7,2.014123e+09,3782265,16000.0,16000.0,16000.0,60 months,11.44%,351.40,B,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
7,8,2.014123e+09,4109512,2500.0,2500.0,2500.0,36 months,11.99%,83.03,B,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
8,9,2.014123e+09,6598721,5250.0,5250.0,5250.0,36 months,11.44%,172.98,B,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
9,10,2.014123e+09,6013917,17000.0,17000.0,17000.0,36 months,13.66%,578.22,C,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Display all columns
pd.set_option('display.max_columns', 200) 

In [6]:
# Column description
df_p2p.describe(include='all')

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
count,235631.000000,2.356310e+05,2.356310e+05,235629.000000,235629.000000,235629.000000,235629,235629,235629.000000,235629,235629,222393,223610,235629,2.356290e+05,235629,235629,235629,235629,0.0,15279,235629,235629,235629,235629,235629.000000,235629.000000,235629,235629.000000,119748.000000,41524.000000,235629.000000,235629.000000,2.356290e+05,235504,235629.000000,235629,235629.000000,235629.000000,235629.000000,235629.000000,235629.000000,235629.000000,2.356290e+05,235629.000000,235629.000000,235486,235629.000000,8241,235603,235629.000000,66478.000000,235629.0,235629,0.0,0.0,0.0,235629.000000,2.356290e+05,2.356290e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.356290e+05,0.0,0.0,0.0,235629.000000,235623.000000,233184.000000,233018.000000,235629.000000,235629.000000,228458.000000,235629.000000,235629.000000,235629.000000,235629.000000,233383.000000,62281.000000,213937.000000,84766.000000,235629.000000,235629.000000,235629.000000,235629.00000,235629.000000,235629.000000,235629.000000,235629.000000,235629.000000,235629.000000,227769.000000,235629.000000,235629.000000,235629.000000,235629.000000,233072.000000,235629.000000,235629.000000,2.356290e+05,2.356290e+05,2.356290e+05,2.356290e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,235629,507,507,507,507.0,507.000000,507,507,507,507.0,507.000000,507,404.000000,507.000000,507.000000,235629,4343,4343,4343,4343.000000,4343.000000,4343.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,2,89,NaN,7,35,75352,11,4,NaN,3,12,6,2,NaN,14843,13,2057,866,49,NaN,NaN,638,NaN,NaN,NaN,NaN,NaN,NaN,1160,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,NaN,2,66,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [7]:
# List of all the columns that will be dropped
dropped_cols = [
    'Unnamed: 0',       # IDs not needed for data analysis
    'id',               # IDs not needed for data analysis
    'member_id',        # IDs not needed for data analysis
    'application_type', # All values homogeneous
    'funded_amnt',      # Values will always be same as loan_amnt
    'url',              # All NaN
    
    # These information below are not available before accepting the borrower, so very useless for training or drawing insights
    'debt_settlement_flag_date', 'settlement_status', 'settlement_date', 'settlement_percentage', 'settlement_term',
    'debt_settlement_flag', 'hardship_last_payment_amount', 'hardship_payoff_balance_amount', 'hardship_loan_status',
    'hardship_dpd', 'hardship_length', 'hardship_start_date', 'hardship_end_date', 'hardship_amount', 'deferral_term',
    'hardship_status', 'hardship_reason', 'hardship_type', 'policy_code', 'recoveries', 'out_prncp_inv', 'out_prncp',
    'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'collection_recovery_fee', 'settlement_amount',
    'payment_plan_start_date'
]

In [8]:
# Drop all the listed columns
df_p2p.drop(axis=1, columns=dropped_cols, inplace=True)

In [9]:
# Drop rows and cols which have all NaN values
df_p2p.dropna(how='all', inplace=True)

In [18]:
# Convert columns to date times, add credit length and secondary app credit length
df_p2p.issue_d = pd.to_datetime(
    df_p2p.issue_d, format='%b-%Y')
df_p2p.earliest_cr_line = pd.to_datetime(
    df_p2p.earliest_cr_line, format='%b-%Y')
df_p2p.sec_app_earliest_cr_line = pd.to_datetime( 
    df_p2p.sec_app_earliest_cr_line, format='%b-%Y')
df_p2p['credit_length'] = df_p2p.issue_d - df_p2p.earliest_cr_line
df_p2p['credit_length'] = df_p2p['credit_length'].dt.days.astype(int)/365
df_p2p['secondary_app_credit_length'] = df_p2p.issue_d - df_p2p.sec_app_earliest_cr_line
df_p2p['secondary_app_credit_length'].fillna(0, inplace=True)
df_p2p['secondary_app_credit_length'] = df_p2p['secondary_app_credit_length'].dt.days.astype(int)/365